In [1]:
import os
import pandas as pd
import numpy as np
import argparse
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../')
from utils import *
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import eig

### hypergraph

In [26]:
df = pd.read_csv('../out/2015.csv_all.csv')

In [27]:
countries = df[df['group'] == 'Country']['name'].unique().tolist()
for country in countries:
    print(country)
    print('centrality = ', df[df['name'] == country]['centrality'].iloc[0], sep='')
    authors = df[df['country'] == country]
    print(authors.sort_values(by='centrality', ascending=False).head())
    print('-'*50)

USA
centrality = 0.9966263284562318
     centrality                     name     group country
166    0.001208             Karlan, Dean  Fullname     USA
175    0.001120       Hammoudeh, Shawkat  Fullname     USA
176    0.001119               Deck, Cary  Fullname     USA
185    0.000968  Bahmani-Oskooee, Mohsen  Fullname     USA
186    0.000964          Fleurbaey, Marc  Fullname     USA
--------------------------------------------------
England
centrality = 7.923511816305432e-05
     centrality                   name     group  country
216    0.000806  Phillips, Peter C. B.  Fullname  England
405    0.000406      Acharya, Viral V.  Fullname  England
673    0.000319      Zodrow, George R.  Fullname  England
692    0.000319         Wohar, Mark E.  Fullname  England
816    0.000243     Santa-Clara, Pedro  Fullname  England
--------------------------------------------------
Germany
centrality = 3.2821348335283974e-05
      centrality                 name     group  country
645     0.000320

2544    0.000159       Davis, John B.  Fullname  Netherlands
--------------------------------------------------
Uganda
centrality = 7.242533640458948e-07
         centrality                      name     group country
2326   1.596355e-04           Wouterse, Fleur  Fullname  Uganda
9985   5.646791e-07     Van Campenhout, Bjorn  Fullname  Uganda
26223  1.621949e-10            Lecoutere, Els  Fullname  Uganda
30755  7.635935e-11  Francesconi, Gian Nicola  Fullname  Uganda
31978  5.783962e-11        Mutamba, Byamah B.  Fullname  Uganda
--------------------------------------------------
New Zealand
centrality = 6.839288609388385e-07
         centrality              name     group      country
3727   8.055678e-05       Claus, Iris  Fullname  New Zealand
8159   7.961709e-05  Turner, James A.  Fullname  New Zealand
10556  3.823539e-08      Davis, Peter  Fullname  New Zealand
12260  7.141725e-09  Scarpa, Riccardo  Fullname  New Zealand
12429  6.566506e-09     Krippner, Leo  Fullname  New Zealan

centrality = 1.9192000176527513e-08
         centrality                             name     group    country
36140  3.065876e-12  Lopez Velasquez, Francisco Jose  Fullname  Venezuela
36745  1.532693e-12                     Gzyl, Henryk  Fullname  Venezuela
36748  1.532693e-12                  Filardo, Leonor  Fullname  Venezuela
--------------------------------------------------
Peru
centrality = 1.9168651782878277e-08
         centrality                        name     group country
13219  6.362472e-09             Larson, Anne M.  Fullname    Peru
13220  6.362472e-09        Cronkleton, Peter J.  Fullname    Peru
35962  4.593830e-12             Escobal, Javier  Fullname    Peru
36144  3.062023e-12  Tellez De Vettori, Giannio  Fullname    Peru
36260  2.553710e-12           Winkelried, Diego  Fullname    Peru
--------------------------------------------------
Madagascar
centrality = 1.9124780984301783e-08
         centrality                           name     group     country
36147  3.

IndexError: single positional indexer is out-of-bounds

### hypergraph with filter: only includes rows(hyperegdes) with specified author-country mapping -> prevent an author to connect multiple organization

In [28]:
df = pd.read_csv('../tmp3.csv')

In [29]:
countries = df[df['group'] == 'Country']['name'].unique().tolist()
for country in countries:
    print(country)
    print('centrality = ', df[df['name'] == country]['centrality'].iloc[0], sep='')
    authors = df[df['country'] == country]
    print(authors.sort_values(by='centrality', ascending=False).head())
    print('-'*50)

USA
centrality = 0.9966625991925518
     centrality                     name     group country
161    0.001260             Karlan, Dean  Fullname     USA
169    0.001167               Deck, Cary  Fullname     USA
170    0.001167       Hammoudeh, Shawkat  Fullname     USA
183    0.001010  Bahmani-Oskooee, Mohsen  Fullname     USA
184    0.001005          Fleurbaey, Marc  Fullname     USA
--------------------------------------------------
England
centrality = 2.0384516202236494e-05
        centrality                      name     group  country
9524  1.324043e-06            Cordella, Tito  Fullname  England
9549  8.822973e-07  Marcellino, Massimiliano  Fullname  England
9551  7.146026e-07            Beck, Thorsten  Fullname  England
9572  5.363304e-07          Ramadorai, Tarun  Fullname  England
9573  5.362769e-07         Valletti, Tommaso  Fullname  England
--------------------------------------------------
Germany
centrality = 1.2562640166700765e-05
        centrality                  

26856  3.503617e-11              Bonilla, Jorge  Fullname  Colombia
--------------------------------------------------
Uruguay
centrality = 2.0898027415305004e-07
         centrality                 name     group  country
9608   2.080566e-07      Crespi, Gustavo  Fullname  Uruguay
22789  1.569051e-10         Aboal, Diego  Fullname  Uruguay
23501  1.045879e-10      Perera, Marcelo  Fullname  Uruguay
24913  6.968651e-11  Lanzilotta, Bibiana  Fullname  Uruguay
26420  5.227360e-11    Gandelman, Nestor  Fullname  Uruguay
--------------------------------------------------
Bolivia
centrality = 2.0812589656551645e-07
         centrality                   name     group  country
9606   2.080912e-07       Stucchi, Rodolfo  Fullname  Bolivia
28268  1.733739e-11  Pablo Chumacero, Juan  Fullname  Bolivia
28270  1.733739e-11          Cortez, Guido  Fullname  Bolivia
--------------------------------------------------
Chile
centrality = 1.770686850399441e-07
        centrality                      na

34180  3.441716e-17        Nasreen, Samia  Fullname  Pakistan
--------------------------------------------------
Slovenia
centrality = 1.5699377806290194e-13
         centrality             name     group   country
31960  1.555091e-13  Stanovnik, Tine  Fullname  Slovenia
33959  7.868511e-17     Kostevc, Crt  Fullname  Slovenia
33960  7.741232e-17    Rojec, Matija  Fullname  Slovenia
34060  5.369178e-17   Bojnec, Stefan  Fullname  Slovenia
34159  3.988082e-17    Polanec, Saso  Fullname  Slovenia
--------------------------------------------------
Norway
centrality = 9.587073798122847e-14
         centrality               name     group country
32044  8.721528e-14  Boler, Esther Ann  Fullname  Norway
32336  5.514251e-15   Nenov, Plamen T.  Fullname  Norway
34092  4.900356e-17  Tungodden, Bertil  Fullname  Norway
34181  3.434603e-17   Rundmo, Torbjorn  Fullname  Norway
34183  3.399969e-17       Iversen, Tor  Fullname  Norway
--------------------------------------------------
Romania
centra

centrality = 1.903703055461897e-18
         centrality             name     group country
36304  1.417064e-18     Agbo, Maxime  Fullname   Benin
36910  7.455973e-19  Nakelse, Tebila  Fullname   Benin
38045  2.210767e-19    Fiamohe, Rose  Fullname   Benin
38105  2.023983e-19    Coulibaly, O.  Fullname   Benin
--------------------------------------------------
Uzbekistan
centrality = 1.5914542590980322e-18
         centrality                 name     group     country
38713  4.701536e-20  Mirkasimov, Bakhrom  Fullname  Uzbekistan
--------------------------------------------------
Cameroon
centrality = 1.5008657508964624e-18
         centrality                  name     group   country
36197  1.718205e-18      Moyouwou, Issofa  Fullname  Cameroon
36932  7.300433e-19   Baye, Francis Menjo  Fullname  Cameroon
37903  2.642323e-19  Balgah, Roland Azibo  Fullname  Cameroon
38462  1.069869e-19      Asongu, Simplice  Fullname  Cameroon
38820  2.231010e-20    Epo, Boniface Ngah  Fullname  Cameroo

IndexError: single positional indexer is out-of-bounds

traditional graph with multiple edges i.e. weight graphs
```python
def getSparseIncidenceMatrixFromId(df, cols, colnum):
    data, row, col = [], [], []
    for x in zip(*[df[col] for col in cols]):
        for i in range(3):
            data.append(1)
            row.append(x[i])
            col.append(x[(i + 1) % 3])

            data.append(1)
            col.append(x[i])
            row.append(x[(i + 1) % 3])

    return csr_matrix((data, (row, col))).asfptype()
```

In [4]:
df = pd.read_csv('../tmp.csv')

In [23]:
countries = df[df['group'] == 'Country']['name'].unique().tolist()
for country in countries:
    print(country)
    print('centrality = ', df[df['name'] == country]['centrality'].iloc[0], sep='')
    authors = df[df['country'] == country]
    print(authors.sort_values(by='centrality', ascending=False).head())
    print('-'*50)

USA
centrality = 0.7060778375892246
     centrality                     name     group country
162    0.011745             Karlan, Dean  Fullname     USA
175    0.010078       Hammoudeh, Shawkat  Fullname     USA
176    0.010075               Deck, Cary  Fullname     USA
182    0.009600  Bahmani-Oskooee, Mohsen  Fullname     USA
183    0.009164          Fleurbaey, Marc  Fullname     USA
--------------------------------------------------
England
centrality = 0.008446710209665908
     centrality                   name     group  country
203    0.007780  Phillips, Peter C. B.  Fullname  England
365    0.004300      Acharya, Viral V.  Fullname  England
673    0.002890      Zodrow, George R.  Fullname  England
686    0.002865         Wohar, Mark E.  Fullname  England
813    0.002522     Santa-Clara, Pedro  Fullname  England
--------------------------------------------------
Germany
centrality = 0.0032095660756770693
      centrality                 name     group  country
647     0.002950  

centrality = 8.40464582649564e-05
         centrality              name     group      country
3799   7.945295e-04       Claus, Iris  Fullname  New Zealand
8352   6.952190e-04  Turner, James A.  Fullname  New Zealand
11372  1.186844e-05      Davis, Peter  Fullname  New Zealand
11638  9.800768e-06  Scarpa, Riccardo  Fullname  New Zealand
18415  9.993317e-07     Krippner, Leo  Fullname  New Zealand
--------------------------------------------------
Uganda
centrality = 7.716355933609974e-05
         centrality                      name     group country
2345   1.431886e-03           Wouterse, Fleur  Fullname  Uganda
10112  6.014332e-05     Van Campenhout, Bjorn  Fullname  Uganda
26764  2.397306e-07            Lecoutere, Els  Fullname  Uganda
31765  1.002467e-07  Francesconi, Gian Nicola  Fullname  Uganda
32278  7.574493e-08        Mutamba, Byamah B.  Fullname  Uganda
--------------------------------------------------
Belgium
centrality = 7.383618077540195e-05
      centrality             

centrality = 2.435802976946153e-06
       centrality                       name     group country
10270    0.000040           Vettas, Nikolaos  Fullname  Greece
10271    0.000040          Genakos, Christos  Fullname  Greece
11425    0.000011  Kanellopoulos, Panagiotis  Fullname  Greece
11830    0.000009     Christodoulakis, Nicos  Fullname  Greece
12059    0.000009          Vlamis, Prodromos  Fullname  Greece
--------------------------------------------------
Mexico
centrality = 2.4244831487041157e-06
         centrality                        name     group country
3502   8.006676e-04  Bernal-Verdugo, Lorenzo E.  Fullname  Mexico
12850  8.602963e-06              Bacaria, Jordi  Fullname  Mexico
18575  9.302692e-07           Gouel, Christophe  Fullname  Mexico
27954  1.987444e-07         Carrasco, Carlos A.  Fullname  Mexico
34557  1.949622e-08            Cypher, James M.  Fullname  Mexico
--------------------------------------------------
Thailand
centrality = 2.2155049905936074e-06
 

IndexError: single positional indexer is out-of-bounds

traditional graph with single edge
```python
def getSparseIncidenceMatrixFromId(df, cols, colnum):
    data, row, col = [], [], []
    exist = set()
    for x in zip(*[df[col] for col in cols]):
        for i in range(3):
            if (x[i], x[(i + 1)%3]) not in exist:
                data.append(1)
                row.append(x[i])
                col.append(x[(i + 1) % 3])
                exist.add((x[i], x[(i + 1)%3]))

            if (x[(i + 1)%3], x[i]) not in exist:
                data.append(1)
                col.append(x[i])
                row.append(x[(i + 1) % 3])
                exist.add((x[(i + 1)%3], x[i]))

    return csr_matrix((data, (row, col))).asfptype()
```

In [24]:
df = pd.read_csv('../tmp2.csv')

In [25]:
countries = df[df['group'] == 'Country']['name'].unique().tolist()
for country in countries:
    print(country)
    print('centrality = ', df[df['name'] == country]['centrality'].iloc[0], sep='')
    authors = df[df['country'] == country]
    print(authors.sort_values(by='centrality', ascending=False).head())
    print('-'*50)

USA
centrality = 0.7010575148775123
     centrality                   name     group country
152    0.008023          Duflo, Esther  Fullname     USA
162    0.007980        Moretti, Enrico  Fullname     USA
163    0.007964  Mitchener, Kris James  Fullname     USA
170    0.007889        Zingales, Luigi  Fullname     USA
171    0.007886       Keller, Wolfgang  Fullname     USA
--------------------------------------------------
England
centrality = 0.01670184527128251
     centrality                name     group  country
182    0.007835      Wei, Shang-Jin  Fullname  England
183    0.007817  Santa-Clara, Pedro  Fullname  England
214    0.007709   Acharya, Viral V.  Fullname  England
245    0.007632   Gaynor, Martin S.  Fullname  England
269    0.007579        Edmans, Alex  Fullname  England
--------------------------------------------------
Germany
centrality = 0.012910580576326264
     centrality               name     group  country
169    0.007927  Becker, Sascha O.  Fullname  Germany

centrality = 0.0005734570118078175
       centrality                name     group country
1818     0.007132   Loyalka, Prashant  Fullname  Russia
6838     0.007042   Tsygankov, Andrei  Fullname  Russia
10956    0.000187    Lokshin, Michael  Fullname  Russia
13556    0.000140  Tarasov, Alexander  Fullname  Russia
18293    0.000076   Kheifets, Igor L.  Fullname  Russia
--------------------------------------------------
Indonesia
centrality = 0.0005220857926303021
       centrality                  name     group    country
1942     0.007127       Minot, Nicholas  Fullname  Indonesia
10962    0.000186   Crystallin, Masyita  Fullname  Indonesia
15938    0.000113        Belcher, Brian  Fullname  Indonesia
18281    0.000076  Achdiawan, Ramadhani  Fullname  Indonesia
18282    0.000076       Moeliono, Moira  Fullname  Indonesia
--------------------------------------------------
Kenya
centrality = 0.0005078130749690886
      centrality                  name     group country
1785    0.007137  

centrality = 0.00014167657178728107
       centrality                             name     group    country
36579    0.000001  Lopez Velasquez, Francisco Jose  Fullname  Venezuela
36582    0.000001                  Filardo, Leonor  Fullname  Venezuela
36584    0.000001                     Gzyl, Henryk  Fullname  Venezuela
--------------------------------------------------
Madagascar
centrality = 0.00014147670577974118
       centrality                           name     group     country
35185    0.000003  Zafindrasilivonona, Bienvenue  Fullname  Madagascar
36585    0.000001         Andriamahefazafy, Fano  Fullname  Madagascar
--------------------------------------------------
Ghana
centrality = 0.00011782603436535459
       centrality                     name     group country
12273    0.000169      Ahiakpor, Ferdinand  Fullname   Ghana
15840    0.000116        Das Gupta, Sunipa  Fullname   Ghana
36647    0.000001         Fosu-Brefo, Rita  Fullname   Ghana
36648    0.000001  Agyemang,

38273  1.774430e-08    Benchekroun, S.  Fullname  Morocco
--------------------------------------------------
Malta
centrality = 1.6986379960716146e-06
         centrality                name     group country
12856  1.676552e-04    Azzopardi, Brian  Fullname   Malta
38277  1.703748e-08  Briguglio, Lino P.  Fullname   Malta
38278  1.703748e-08     Vella, Melchior  Fullname   Malta
--------------------------------------------------
nan


IndexError: single positional indexer is out-of-bounds